# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, observables=[])

In [3]:
# as dict
dict(vocs)

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'observables': []}

In [4]:
#  re-parse dict
vocs2 = VOCS.from_dict(dict(vocs))

In [5]:
# Check that these are the same
vocs2 == vocs

True

In [6]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [7]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [8]:
# json
vocs.to_json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"observables":[]}'

# Objective Evaluation

In [9]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,457.389102,-0.358454,1234,457.030648,457.747555,914.061296,915.495111
6,909.170245,-0.010033,1234,909.160212,909.180278,1818.320425,1818.360555
7,422.776529,-0.677400,1234,422.099130,423.453929,844.198259,846.907858
8,454.479496,-0.870863,1234,453.608632,455.350359,907.217264,910.700718
9,4.632875,0.255194,1234,4.888069,4.377681,9.776139,8.755363
10,278.216342,-0.254661,1234,277.961681,278.471003,555.923363,556.942006
11,138.994960,0.163017,1234,139.157977,138.831944,278.315954,277.663887
12,963.238898,-0.741689,1234,962.497210,963.980587,1924.994420,1927.961174
13,673.537035,-0.543064,1234,672.993971,674.080099,1345.987942,1348.160198
14,67.162800,-0.899678,1234,66.263122,68.062478,132.526244,136.124956


In [10]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [11]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-457.030648,457.747555
6,-909.160212,909.180278
7,-422.099130,423.453929
8,-453.608632,455.350359
9,-4.888069,4.377681
10,-277.961681,278.471003
11,-139.157977,138.831944
12,-962.497210,963.980587
13,-672.993971,674.080099
14,-66.263122,68.062478


In [12]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-457.030648,457.747555
6,-909.160212,909.180278
7,-422.099130,423.453929
8,-453.608632,455.350359
9,-4.888069,4.377681
10,-277.961681,278.471003
11,-139.157977,138.831944
12,-962.497210,963.980587
13,-672.993971,674.080099
14,-66.263122,68.062478


In [13]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-457.03064787,  457.74755543],
       [-909.16021234,  909.18027752],
       [-422.09912972,  423.45392906],
       [-453.60863224,  455.35035884],
       [  -4.88806927,    4.37768142],
       [-277.96168136,  278.47100295],
       [-139.15797679,  138.83194372],
       [-962.49720987,  963.98058709],
       [-672.99397107,  674.08009891],
       [ -66.26312196,   68.06247775]])

In [14]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,912.061296,-915.495111
6,1816.320425,-1818.360555
7,842.198259,-846.907858
8,905.217264,-910.700718
9,7.776139,-8.755363
10,553.923363,-556.942006
11,276.315954,-277.663887
12,1922.994420,-1927.961174
13,1343.987942,-1348.160198
14,130.526244,-136.124956


In [15]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


In [16]:
# normalize inputs to unit domain [0,1]
vocs.normalize_inputs(data)

,a,b
5,0.457389,0.320773
6,0.909170,0.494984
7,0.422777,0.161300
8,0.454479,0.064568
9,0.004633,0.627597
10,0.278216,0.372670
11,0.138995,0.581508
12,0.963239,0.129156
13,0.673537,0.228468
14,0.067163,0.050161


# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
